03. DATA PREPARATION

In [ ]:
import pandas as pd
import numpy as np

filepath = '../../data/coords/old10s-60f.csv'

df = pd.read_csv(filepath)

df.columns

Create good subsets of frames

In [ ]:
# Exclude the frame index column (assume it's the first column)
data_cols = df.columns[1:]

# Count non-empty (non-NaN) cells per row
df['detection_count'] = df[data_cols].notnull().sum(axis=1)

# Find indices of good frames
good_mask = df['detection_count'] > 18

# Find all consecutive subsets of good frames (no min length)
subsets = []
start = None
for idx, is_good in enumerate(good_mask):
    if is_good:
        if start is None:
            start = idx
    else:
        if start is not None:
            subsets.append((start, idx))
            start = None
# Handle sequence that goes to the end
if start is not None:
    subsets.append((start, len(df)))

# Save each subset as a CSV
import os
os.makedirs('../../data/data_preparation/subsets', exist_ok=True)
for i, (start, end) in enumerate(subsets):
    if end - start >= 10:
        subset = df.iloc[start:end]
        subset.to_csv(f'../../data/data_preparation/subsets/subset{i+1}.csv', index=False)

See Transitions of ids or detect ids of referees or other entities - manually

In [ ]:
#drop empty columns
df = df.dropna(axis=1, how='all')

#drop extra columns
df = df.drop(columns=['detection_count'])

#IDs to remove
#ids: 11 - referee, 24 - referee
df = df.drop(columns=['id_11_team_1'])
df = df.drop(columns=['id_24_team_1'])
df = df.drop(columns=['gk0'])
df = df.drop(columns=['gk1'])
df = df.rename(columns={'id_23_team_1': 'gk0'})

#IDs with no Transitions
#ids:1,3,4,5,6,9,10,12,13,14,15,19,22

#IDs with Transitions
#ids: 2,7,8,16,17,18

#id2 lt f14 -> id43
df.loc[14:, 'id_2_team_1'] = df.loc[14:, 'id_43_team_1']
df = df.drop(columns=['id_43_team_1'])

#id7 lt f3-6 -> id 29
df.loc[7:, 'id_7_team_1'] = df.loc[7:, 'id_29_team_1']
df = df.drop(columns=['id_29_team_1'])

#id8 lt f9 -> id34
df.loc[9:, 'id_8_team_1'] = df.loc[9:, 'id_34_team_1']
df = df.drop(columns=['id_34_team_1'])

#id16 lt f20 -> id53
df.loc[20:, 'id_16_team_0'] = df.loc[20:, 'id_53_team_0']
df = df.drop(columns=['id_53_team_0'])

#id17 lt f5-6 -> id30 lt f8-12 -> id39
df.loc[6:, 'id_17_team_1'] = df.loc[6:, 'id_30_team_1']
df.loc[12:, 'id_17_team_1'] = df.loc[12:, 'id_39_team_1']
df = df.drop(columns=['id_30_team_1', 'id_39_team_1'])

#id18f1 lt f2 -> id18 lt f9-10 -> id36 lt f12-15 -> id44
df.loc[10:, 'id_18_team_1'] = df.loc[10:, 'id_36_team_1']
df.loc[15:, 'id_18_team_1'] = df.loc[15:, 'id_44_team_1']
df = df.drop(columns=['id_36_team_1', 'id_44_team_1'])

#id18f2 lt f3-5 -> id28
df.loc[df['frame_index'] == 2, 'id_28_team_0'] = df.loc[df['frame_index'] == 2, 'id_18_team_1']
df.loc[df['frame_index'] == 2, 'id_18_team_1'] = np.nan

#save cleaned subset
#df.to_csv(subset, index=False)

Interpolate good subsets

In [ ]:
#not working
import pandas as pd

df = pd.read_csv('../../data/train/ss1.csv')

# Convert all columns except the index (if any) to numeric, coercing errors to NaN
for col in df.columns:
    if col != 'frame':  # replace 'frame' with your index column if needed
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Interpolate all numeric columns (coordinates)
df.interpolate(method='linear', limit_direction='both', inplace=True)

# Save the interpolated DataFrame
df.to_csv('../../data/train/ss1_interpolated.csv', index=False)